In [34]:
%matplotlib notebook
import dill
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

In [2]:

filename = '/Users/bpb/Downloads/positive_mode_isotope_data.pkl'
with open(filename,'rb') as fid:
    data = dill.load(fid)

In [3]:
[(i,d['compound']) for i,d in enumerate(data)]

[(0, u'glycocyamine_10p54'),
 (1, u'D-Glucosamine_10p77'),
 (2, u'L-Allothreonine_10p03'),
 (3, u'creatinine_4p51'),
 (4, u'N-Acetylneuraminic_9p74'),
 (5, u'2-Aminoisobutyric_9p24'),
 (6, u'1-Aminocyclopropanecarboxylic_10p01'),
 (7, u'dl-3-Aminoisobutyric_10p08'),
 (8, u'5-Aminovaleric_11p98'),
 (9, u'L-ornithine_18p53'),
 (10, u'xanthosine_8p45'),
 (11, u'L-2-Aminoadipic_11p82'),
 (12, u'2deoxycytidine_5p30'),
 (13, u'L-arabitol_6p30'),
 (14, u'14289-34-0_14p73'),
 (15, u'3-carboxy-1-methylpyridinium_8p47'),
 (16, u'L-glutamine_10p90'),
 (17, u'L-threonine_10p51'),
 (18, u'Imidazole-4-acetic_9p83'),
 (19, u'2deoxyadenosine_2p25'),
 (20, u'creatine_10p23'),
 (21, u'ectoine_9p56'),
 (22, u'L-tyrosine_8p74'),
 (23, u'cytidine_6p58'),
 (24, u'L-citrulline_11p53'),
 (25, u'nicotinamide_1p42'),
 (26, u'2deoxyguanosine_6p36'),
 (27, u'adenosine_3p52'),
 (28, u'4-aminobutyric_11p66'),
 (29, u'L-methionine_7p78'),
 (30, u'L-lysine_18p40'),
 (31, u'cytosine_5p23'),
 (32, u'TRIMETHYL_8p17'),
 

In [4]:
cpd_id = 30

In [5]:
len(data)

61

In [6]:
data[cpd_id].keys()

['rt_peak', 'peak_height', 'compound']

In [7]:
data[cpd_id]['compound']

u'L-lysine_18p40'

In [8]:
data[cpd_id]['rt_peak'].columns[:3]

Index([u'20171221_TS_DB_fullcyc_QE144_50454-738379_30_13C_Cel_D30_2_POS_106.h5',
       u'20171221_TS_DB_fullcyc_QE144_50454-738379_74_13C_Van_D48_3_POS_172.h5',
       u'20171221_TS_DB_fullcyc_QE144_50454-738379_94_13C_Ami_D14_1_POS_221.h5'],
      dtype='object', name=u'file')

In [9]:
data[cpd_id]['peak_height'].columns[:3]

Index([u'20171221_TS_DB_fullcyc_QE144_50454-738379_30_13C_Cel_D30_2_POS_106.h5',
       u'20171221_TS_DB_fullcyc_QE144_50454-738379_74_13C_Van_D48_3_POS_172.h5',
       u'20171221_TS_DB_fullcyc_QE144_50454-738379_94_13C_Ami_D14_1_POS_221.h5'],
      dtype='object', name=u'file')

In [29]:
treatments = list(set(['_'.join(c.split('_')[7:9]) for c in data[cpd_id]['peak_height'].columns if 'D48' in c]))
treatments

[u'13C_Cel', u'12C_con', u'13C_Pal', u'13C_Van', u'H2O_con']

In [42]:
time_points = []
for c in data[cpd_id]['peak_height'].columns:
    try:
        found = re.search('_D\d+_', c).group()
        time_points.append(found.strip('_'))
    except AttributeError:
        pass
time_points = list(set(time_points))
time_points

[u'D14', u'D30', u'D48', u'D6', u'D3', u'D1']

In [10]:
groups = list(set(['_'.join(c.split('_')[7:10]) for c in data[cpd_id]['peak_height'].columns if 'D48' in c]))
groups

[u'12C_con_D48',
 u'13C_Pal_D48',
 u'13C_Cel_D48',
 u'H2O_con_D48',
 u'13C_Van_D48']

In [11]:
peak_height = data[cpd_id]['peak_height']
control = peak_height[[c for c in peak_height.columns if groups[0] in c]].mean(axis=1)
control

mz
147p1128    870130.437500
148p1162     56566.234375
149p1195      5923.273315
150p1229      1014.861206
151p1262         0.000000
152p1296     16176.509766
153p1329      1269.669434
dtype: float64

In [12]:
labeled = peak_height[[c for c in peak_height.columns if groups[2] in c]].mean(axis=1)
labeled

mz
147p1128    738767.677083
148p1162     52596.792969
149p1195      9509.240885
150p1229      1036.475342
151p1262      1257.195516
152p1296     16633.710938
153p1329       743.332275
dtype: float64

In [13]:
labeled - control

mz
147p1128   -131362.760417
148p1162     -3969.441406
149p1195      3585.967570
150p1229        21.614136
151p1262      1257.195516
152p1296       457.201172
153p1329      -526.337158
dtype: float64

In [14]:
diff_data = {}
for g in groups:
    sub_data = peak_height[[c for c in peak_height.columns if g in c]].mean(axis=1)
    diff_data[g] = sub_data
df = pd.DataFrame(diff_data)
df

,12C_con_D48,13C_Cel_D48,13C_Pal_D48,13C_Van_D48,H2O_con_D48
mz,,,,,
147p1128,870130.437500,738767.677083,1.017697e+06,807535.562500,482432.500000
148p1162,56566.234375,52596.792969,7.030222e+04,51074.240885,34027.096354
149p1195,5923.273315,9509.240885,1.283069e+04,6811.392171,5410.415771
150p1229,1014.861206,1036.475342,9.736336e+02,0.000000,0.000000
151p1262,0.000000,1257.195516,0.000000e+00,1351.055013,2449.941243
152p1296,16176.509766,16633.710938,1.615290e+04,15990.358724,22510.484375
153p1329,1269.669434,743.332275,0.000000e+00,864.773844,1387.787109


In [48]:
plt.ioff()
for time_point in time_points:
    folder_name = 'isotope_figs'
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    for cpd_id in range(len(data)):
        peak_height = data[cpd_id]['peak_height']
        diff_data = {}
        fig = plt.figure()
        ax = fig.gca()
        for treatment in treatments:
            g = '%s_%s'%(treatment,time_point)
            sub_data = peak_height[[c for c in peak_height.columns if g in c]].mean(axis=1)
            diff_data[g] = sub_data
        df = pd.DataFrame(diff_data)
        df.index = [d.replace('p','.') for d in df.index]
        df[:-1].plot(kind='bar',ax=ax) #drop the unnecessary last ion
        plt.title(data[cpd_id]['compound'],fontsize=18)
        ax.set_yscale('log')
        plt.ylabel('Peak Height (au)')
        plt.xlabel('m/z')
        plt.tight_layout()
        fig.savefig(os.path.join(folder_name,'%s_%s.pdf'%(data[cpd_id]['compound'],time_point)))
        fig.clear()
plt.ion()